In [1]:
# # general imports 
import os
from pathlib import Path
import numpy as np
import pandas as pd 

# nwb specific imports 
import pynwb
from nwbwidgets import nwb2widget
from hdmf_zarr import NWBZarrIO 

In [2]:
# Load metadata CSV 

# metadata = pd.read_csv('/data/metadata/V1DD_metadata.csv') # V1DD 
# metadata = pd.read_csv('/data/metadata/dynamic_foraging_metadata.csv') # Dynamic Foraging 
metadata = pd.read_csv('/data/metadata/bci_metadata.csv') # BCI 
metadata.head()

,_id,name,subject_id,genotype,virus_injection_materials,date_of_birth,sex,session_type,session_time,stimulus_epochs,project_name,modality,targeted_structure,ophys_fov,session_number
0,03f1b82e-4494-46bc-a959-50c4b14d13da,single-plane-ophys_731015_2025-01-28_17-40-57_...,731015,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,2024-03-14,Female,BCI single neuron stim,2025-01-28T17:40:57.996000-08:00,"['spontaneous activity', 'spontaneous activity...",Brain Computer Interface,['Planar optical physiology'],['Primary Motor Cortex'],FOV_04,22.0
1,b28d6321-1b1b-4110-87fb-0e19325c1dfe,single-plane-ophys_767715_2025-02-17_17-41-50_...,767715,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,2024-09-15,Female,BCI single neuron stim,2025-02-17T17:41:50.758000-08:00,"['spontaneous activity', 'spontaneous activity...",Brain Computer Interface,['Planar optical physiology'],['Primary Motor Cortex'],FOV_03,8.0
2,580448b2-4b93-4477-b6ac-6e2972792fe0,single-plane-ophys_772414_2025-01-27_15-57-05_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,BCI single neuron stim,2025-01-27T14:23:26.693000-08:00,"['spontaneous activity', 'spontaneous activity...",Brain Computer Interface,['Planar optical physiology'],['Primary Motor Cortex'],FOV_01,5.0
3,c1be8449-7bef-4967-88f5-6a6ffc29e5a3,single-plane-ophys_772414_2025-02-06_16-47-44_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,BCI single neuron stim,2025-02-06T16:47:44.152999-08:00,"['spontaneous activity', 'spontaneous activity...",Brain Computer Interface,['Planar optical physiology'],['Primary Motor Cortex'],FOV_02,13.0
4,15b395c0-1caf-4be4-a212-a13b9a4d1dfc,single-plane-ophys_772414_2025-02-10_11-15-26_...,772414,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,2024-10-15,Female,BCI single neuron stim,2025-02-10T11:15:26.609999-08:00,"['spontaneous activity', 'spontaneous activity...",Brain Computer Interface,['Planar optical physiology'],['Primary Motor Cortex'],FOV_03,14.0


In [3]:
metadata.columns

Index(['_id', 'name', 'subject_id', 'genotype', 'virus_injection_materials',
       'date_of_birth', 'sex', 'session_type', 'session_time',
       'stimulus_epochs', 'project_name', 'modality', 'targeted_structure',
       'ophys_fov', 'session_number'],
      dtype='object')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** What modality was used?
</div>

In [4]:
metadata.modality.unique()

array(["['Planar optical physiology']"], dtype=object)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** How many mice were used in this dataset?
</div>

In [5]:
len(metadata.subject_id.unique())

6

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** How many sessions? How many sessions per mouse? 
</div>

In [7]:
len(metadata.name.unique())

33

In [8]:
metadata['subject_id'].value_counts()

subject_id
731015    7
740369    7
767715    6
754303    6
772414    4
766719    3
Name: count, dtype: int64

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** What transgenic lines were used in this dataset? 
    
What cell types do these transgenic lines target? Refer to the [Data Book](https://allenswdb.github.io/background/transgenic-tools.html) for this. 
</div>

In [10]:
metadata.genotype.unique()

array(['Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-ICL-ChRmine-oScarlet-IRES2-tTA2-WPRE)-hyg/wt',
       'Camk2a-tTA/wt;tetO-GCaMP6s/wt',
       'Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt'], dtype=object)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise:** What structures were targeted? If not in metadata, might need to infer from Data Book or NWB file itself. 

In [77]:
metadata.targeted_structure.unique()

array(["['Primary Motor Cortex']"], dtype=object)

In [26]:
# # Load session data 

# note issue with loading BCI data using prev method as it doesn't end in .nwb 
# just copy the path for a sample session 
nwb_path_zarr = '/data/brain-computer-interface/single-plane-ophys_767715_2025-02-17_17-41-50_processed_2025-08-05_01-05-20/single-plane-ophys_767715_2025-02-17_17-41-50_behavior_nwb'

In [30]:
io = NWBZarrIO(nwb_path_zarr, mode = 'r') 
nwbfile_zarr = io.read()

/opt/conda/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.7.0 because version 2.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [32]:
nwb2widget(nwbfile_zarr)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: What units/neurons are available? How many neurons were recorded in this session? Has the data been pre-filtered based on quality control metrics? 

In [70]:
image_segmentation = nwbfile_zarr.processing['processed'].data_interfaces['image_segmentation'].plane_segmentations['roi_table'].to_dataframe()
image_segmentation

,is_soma,soma_probability,is_dendrite,dendrite_probability,image_mask
id,,,,,
0,1,0.757573,0,3.039837e-06,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,0,0.115644,0,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,0,0.249087,0,7.450581e-07,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,0,0.073840,0,9.238720e-07,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,0,0.476796,0,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...
2307,0,0.000000,0,1.221895e-06,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2308,0,0.000000,0,1.260638e-05,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2309,0,0.000000,1,9.949803e-01,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [71]:
image_segmentation.is_soma.unique()

array([1, 0])

In [72]:
len(image_segmentation[image_segmentation.is_soma == 1])

221

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: What stimuli, behaviors, and conditions were applied or recorded in this experiment? 

In [79]:
epochs = nwbfile_zarr.intervals['epochs'].to_dataframe()
epochs.stimulus_name.unique()

array(['photostim', 'spont', 'BCI', 'spont_post', 'photostim_post'],
      dtype=object)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: How long is a session? 

In [82]:
epochs['stop_time'].iloc[4]/60

np.float64(68.27870292932538)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: What is the sampling rate? 

In [85]:
nwbfile_zarr.processing['processed'].data_interfaces['dff'].roi_response_series['dff'].rate

58.2262

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Exercise**: Did the mice receive behavior training before recording sessions? Are the sessions longitudinal? 

Multiple recording sessions per mouse (longitudinal) 